# Super DTW speech Recognizer

### Librerias

In [19]:
import os
import numpy as np
import scipy.spatial.distance as dist
from pydub import AudioSegment
from pydub.silence import split_on_silence

# Audio process 
from scipy.io import wavfile
import IPython.display as ipyd
import librosa
import librosa.display
import warnings
warnings.filterwarnings('ignore')

In [4]:
import sounddevice as sd
from scipy.io.wavfile import write
import wavio as wv
  
freq = 16000  
duration = 7
print('Escuchando...')
recording = sd.rec(int(duration * freq), 
                   samplerate=freq, channels=1)  
sd.wait()
input_file2test = './audioInput.wav'
wv.write(input_file2test, recording, freq, sampwidth=2)

listenign


In [10]:
# input_file2test = 'src/coversor/files2conver/bot.wav' # Archivo a procesar
# input_file2test = 'audios/tests/mike_test.wav' # MAd mike
sound_file2test = AudioSegment.from_wav(input_file2test) # Aqui su archivo en WAV

### Algoritmo DTW

In [6]:
def dp(dist_mat):
    N, M = dist_mat.shape
    
    # Initialize the cost matrix
    cost_mat = np.zeros((N + 1, M + 1))
    for i in range(1, N + 1):
        cost_mat[i, 0] = np.inf
    for i in range(1, M + 1):
        cost_mat[0, i] = np.inf

    # Fill the cost matrix while keeping traceback information
    traceback_mat = np.zeros((N, M))
    for i in range(N):
        for j in range(M):
            penalty = [
                cost_mat[i, j],      # match (0)
                cost_mat[i, j + 1],  # insertion (1)
                cost_mat[i + 1, j]]  # deletion (2)
            i_penalty = np.argmin(penalty)
            cost_mat[i + 1, j + 1] = dist_mat[i, j] + penalty[i_penalty]
            traceback_mat[i, j] = i_penalty

    # Traceback from bottom right
    i = N - 1
    j = M - 1
    path = [(i, j)]
    while i > 0 or j > 0:
        tb_type = traceback_mat[i, j]
        if tb_type == 0:
            # Match
            i = i - 1
            j = j - 1
        elif tb_type == 1:
            # Insertion
            i = i - 1
        elif tb_type == 2:
            # Deletion
            j = j - 1
        path.append((i, j))

    # Strip infinity edges from cost_mat before returning
    cost_mat = cost_mat[1:, 1:]
    return (path[::-1], cost_mat)

### Metodo para la obtencion de specs de los splits

In [7]:
# Audio a procesar
# Primero extraemos el espectrograma 
# query_filename = "src/coversor/convertedFiles/hello1.wav" #Archivo a comparar

def input_spec(query_fn):
    f_s, x = wavfile.read(query_fn)
    #------ the hop length
    n_fft = int(0.025*f_s)      # 25 ms
    hop_length = int(0.01*f_s)  # 10 ms
    # Mel-scale spectrogram
    mel_spec_x = librosa.feature.melspectrogram(
    y=x/1.0, sr=f_s, n_mels=40,
    n_fft=n_fft, hop_length=hop_length
    )
    log_mel_spec_x = np.log(mel_spec_x)
    x_seq = log_mel_spec_x.T
    # ipyd.Audio(rate=f_s, data=x)
    return f_s, x, n_fft ,x_seq, hop_length

### División de Audio original de entrada en segmentos

In [27]:
audio_chunks = split_on_silence(sound_file2test, min_silence_len=200, silence_thresh=-30)
# Los dividimos en segmentos en WAV -> mono canal y 16 bits de rate 
# print('The lem: ',len(audio_chunks), type(audio_chunks))
for i, chunk in enumerate(audio_chunks):
   sound = chunk
   out_file = "splits/chunk{0}.wav".format(i)
   # Here concert the audio to mono.
   # sound = AudioSegment.from_mp3(source)  # load source
   print("Exportando: ", out_file)
   sound = sound.set_channels(1)  # mono   
   sound = sound.set_frame_rate(16000)  # 16000Hz
   # print(f'The audio segment channel: {sound.channels}')
   sound.export(out_file, format="wav") 

Exportando:  splits/chunk0.wav
Exportando:  splits/chunk1.wav
Exportando:  splits/chunk2.wav
Exportando:  splits/chunk3.wav
Exportando:  splits/chunk4.wav


In [28]:
audio_folder = 'templates/audios'
# src\coversor\convertedFiles

## Lectura unica de templates

In [29]:
audio_files = os.listdir(audio_folder)  # El escaneador de archivos

data_per_templates = np.array([], dtype=object)
for neighbour_fn in audio_files:
    composePath = audio_folder + '/' + neighbour_fn
    f_s, y = wavfile.read(composePath)
    n_fft = int(0.025*f_s)      # 25 ms
    hop_length = int(0.01*f_s)  # 10 ms
    
    mel_spec_y = librosa.feature.melspectrogram(
        y=y/1.0, sr=f_s, n_mels=40,
        n_fft=n_fft, hop_length=hop_length
    )
    log_mel_spec_y = np.log(mel_spec_y)
    y_seq = log_mel_spec_y.T
    
    separator = '.'
    word_label = neighbour_fn.split(separator, 1)[0]

    data_per_templates = np.append(
        data_per_templates, {"palabra_t": word_label, "datos_spec": y_seq})  

In [21]:
# data_per_templates

#### Comparacion por DTW de split[i] ~= templates[j]

In [35]:
splited_input = './splits'
palabras_reconocidas = []
audio_input_files = os.listdir(splited_input)
for input_fn in audio_input_files:
    path_x = splited_input +'/'+ input_fn
    # sound_file = AudioSegment.from_wav(path_x) # Aqui su archivo en WAV
    f_s_x, x, n_fft ,x_seq, hop_length = input_spec(path_x) # Extraendo el espec del audio(palabra) a analizar

    # Aqui comparamos uno por uno  
    min_values = np.array([], dtype=object)
    # for i, template in np.ndenumerate(data_per_templates):
    for template in data_per_templates:
        y_data_spec = template.get('datos_spec')
        dist_mat = dist.cdist(x_seq, y_data_spec, "cosine")
        path, cost_mat = dp(dist_mat)
        # print("Alignment cost: {:.6f}".format(cost_mat[-1, -1]))
        M = y_data_spec.shape[0]
        N = x_seq.shape[0]
        # print(f'Tamaños comparados: M->{M}; N->{N}')
        # print(f'FileName: {neighbour_fn}')
        # print(
        # "Normalized alignment cost: {:.8f}".format(
        # cost_mat[-1, -1]/(M + N))
        # )
        min_values = np.append(min_values,{"palabra_id":template.get('palabra_t') ,"split_":input_fn,"costo":cost_mat[-1, -1]/(M + N)})
        # Here 
        minPricedItem = min(min_values, key=lambda x:x['costo'])

    palabras_reconocidas.append(minPricedItem)     
    

In [36]:
palabras_reconocidas
# len(palabras_reconocidas)

[{'palabra_id': 'el', 'split_': 'chunk0.wav', 'costo': 0.010110092966510432},
 {'palabra_id': '7', 'split_': 'chunk1.wav', 'costo': 0.010236806256244384},
 {'palabra_id': 'el', 'split_': 'chunk2.wav', 'costo': 0.00893407324268928},
 {'palabra_id': 'aula', 'split_': 'chunk3.wav', 'costo': 0.008161790319128863},
 {'palabra_id': '7', 'split_': 'chunk4.wav', 'costo': 0.011840953090360777}]